# Singleton

## 1. 自定义类装饰器

参考:
- https://stackoverflow.com/questions/31875/is-there-a-simple-elegant-way-to-define-singletons

In [5]:
class singleton(object):
    def __init__(self, decorated):
        self._decorated = decorated
    
    def Instance(self):
        try:
            return self._instance
        except AttributeError:
            self._instance = self._decorated()
            return self._instance
    
    def __call__(self, *args, **kwargs):
        raise TypeError('Singleton must be accessed via `Instance`')
    
    def __instancecheck__(self, inst):
        return isinstance(inst, self._decorated)


@singleton
class Foo(object):
    def __init__(self):
        print 'Foo created'
    
    def hello(self):
        print 'Hello, world'


f1 = Foo.Instance()
f2 = Foo.Instance()
print f1 is f2
f1.hello()

Foo created
True
Hello, world


## 2. 重载`__new__`

参考:
- https://stackoverflow.com/questions/31875/is-there-a-simple-elegant-way-to-define-singletons

In [1]:
class TsinghuaUniversity(object):
    _instance = None
    def __new__(cls, pos, age):
        if cls._instance is None:
            cls._instance = \
                super(TsinghuaUniversity, cls).__new__(cls)
            cls._instance.pos = pos
            cls._instance.age = age
        return cls._instance
    
    def __str__(self):
        return 'Tsinghua University'


tsinghua1 = TsinghuaUniversity('beijing', 110)
tsinghua2 = TsinghuaUniversity('tianjin', 120)

print tsinghua1 is tsinghua2
print tsinghua1.pos, tsinghua1.age
print tsinghua2.pos, tsinghua2.age

True
beijing 110
beijing 110


## 3. 使用内部类

参考:
- http://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html

This approach doesn’t restrict you to creating only one object. This is also a technique to create a limited pool of objects.

In [31]:
class Foo(object):
    class __singleton(object):
        def __init__(self, name):
            self.name = name
        
        def hello(self):
            return 'I am the only one'
        
        def __str__(self):
            return 'Foo name=%s' % self.name

    _instance = None
    
    def __init__(self, name):
        if Foo._instance is None:
            Foo._instance = Foo.__singleton(name)
        else:
            Foo._instance.name = name
    
    def __getattr__(self, name):
        return getattr(self._instance, name)


f1 = Foo('dog')
print f1.name
print f1.hello()

f2 = Foo('cat')
print f2.name
print f2.hello()

print f1 is f2
print f1._instance is f2._instance

dog
I am the only one
cat
I am the only one
False
True


## 4. 使用`Borg`

参考: 
- http://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html

不是真正意义上的单体, 而是让所有的实例共享状态

In [36]:
class Borg(object):
    _shared_state = {}
    
    @classmethod
    def singletonize(cls, obj):
        obj.__dict__ = cls._shared_state


class Foo(object):
    def __init__(self, name):
        Borg.singletonize(self)
        self.name = name
    
    def __str__(self):
        return 'Foo name=%s' % self.name


f1 = Foo('aaa')
f2 = Foo('bbb')

print id(f1)
print id(f2)

print f1.name
print f2.name

4376557648
4375656016
bbb
bbb
